In [0]:
from __future__ import absolute_import, division, print_function, unicode_literals

import numpy as np
import tensorflow as tf
#import pathlib
#import matplotlib.pyplot as plt

#from tensorflow.keras.models import Sequential
#from tensorflow.keras.layers import Dense, Conv3D, Flatten, Dropout, MaxPooling3D

#AUTOTUNE = tf.data.experimental.AUTOTUNE

#print("Tensorflow Version: ", tf.__version__)
#print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

In [ ]:
data_dir = pathlib.Path("./data")

partition = {
    'train': [],
    'test': []
}
partition['train'] = (data_dir / "Train").glob("**/*.mp4")
partition['test'] = (data_dir / "Test").glob("**/*.mp4")

In [ ]:
BATCH_SIZE = 4
SHUFFLE_BUFFER_SIZE = 100

train_dataset = train_dataset.shuffle(SHUFFLE_BUFFER_SIZE).batch(BATCH_SIZE)
test_dataset = test_dataset.batch(BATCH_SIZE)
val_dataset = val_dataset.batch(BATCH_SIZE)

In [ ]:
model = Sequential([
    Conv3D(16, (1, 4, 3), activation='relu', input_shape=(15, 240, 320, 3)),
    MaxPooling3D(),
    Conv3D(32, (1, 4, 3), activation='relu'),
    MaxPooling3D(),
    Conv3D(64, (1, 4, 3), activation='relu'),
    MaxPooling3D(),
    Flatten(),
    Dense(512, activation='relu'),
    Dense(2, activation='softmax')
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

model.summary()

In [ ]:
callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

history = model.fit(train_dataset, epochs=100, callbacks=[callback], validation_data=val_dataset)
model.evaluate(test_dataset)

In [ ]:
plt.plot(history.history['accuracy'], label='accuracy')
plt.plot(history.history['val_accuracy'], label = 'val_accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.ylim([0.5, 1])
plt.legend(loc='lower right')

test_loss, test_acc = model.evaluate(test_dataset, verbose=2)

In [ ]:
export_path = pathlib.Path("export/model.h5")
print("Exporting trained model to {}".format(export_path.absolute()))

model.save(export_path)